## Set up `GenX` Project in Current Folder

This step assumes you've installed `julia` already but have not set up the current `GenX` project folder.

In [ ]:
!julia --project=. Install.jl

## Export `GenX` Input CSVs from Spreadsheet

In [ ]:
import shutil

import xlwings as xw
import pandas as pd
from collections import defaultdict
import os
from upath import UPath
from loguru import logger
import sys
from datetime import datetime
from runner import *

logger.remove()
logger.add(sys.stderr, backtrace=False)

wb = xw.Book(
    '/Users/roderick/Library/CloudStorage/OneDrive-SharedLibraries-ResilientTransition/5.001 Kentucky Resource Council - Documents/Data/Kentucky Load Resource Model.xlsb'
)

- [x] Do the xlwings thing where the SharePoint path is updated in the spreadsheet -> used VBA UDF
- [ ] Do we need something like `kit-ui connect`? I don't think so
- [ ] File picker to connect to spreadsheet?
- [x] Format this notebook before committing
- [x] Copy Run.jl
- [x] Copy settings
- [ ] Check that demand_data, fuel_data, and generator_variability CSVs have the same length
- [x] Save a mapping of the planning period to year, so that it can be read in by spreadsheet, or when concatenating the dataframe (instead of guessing)

# Run

In [ ]:
def update_params_and_save_case(*, params: tuple, wb: xw.Book):
    logger.info(f"Updating params: {params}")
    for param, value in params.items():
        wb.sheets["GenX Settings"].range(param).value = value
    
    base_folder = save_multistage_case(wb=wb)
    return base_folder

def run_and_load_case(*, wb: xw.Book):
    import subprocess
    base_folder = save_multistage_case(wb=wb)
    subprocess.run(["julia", "--project=.", "Run.jl", base_folder])
    load_case_results(wb=wb, base_folder=base_folder)

# def update_params_and_run(*, params: tuple, wb: xw.Book):
#     """Update params on GenX Settings and run the GenX case."""
#     logger.info(f"Updating params: {params}")
#     for param, value in params.items():
#         wb.sheets["GenX Settings"].range(param).value = value
    
#     wb.sheets["GenX Results"].api.autofilter_mode.set(False)
#     run_and_load_case(wb=wb)

#     report_wb = xw.Book()
#     wb.sheets["GenX Results"].copy(after=report_wb.sheets[0])
#     report_wb.save(UPath(wb.sheets["GenX Settings"].range("BaseFolder").value) / "results" / "figures.xlsm")
#     report_wb.close()

In [ ]:
params = {
    # "EnablePCM": [False],
    # "IncludePlannedResources": [False],
    # "AllowEconomicRetirements": [True],
    "IncludePTC": ["IRA"], #, "BBB"],
    # "AEO": [2023],
    "NoNewGasYear": [2040],
    "MaxWind": [1000],
    # "FlexReserveReq": [0.06],
}

import itertools
combinations = [dict(zip(params.keys(), v)) for v in itertools.product(*params.values())]

# addl_params = {
#     # "EnablePCM": [False],
#     # "IncludePlannedResources": [False],
#     # "AllowEconomicRetirements": [False],
#     "IncludePTC": ["IRA", "BBB"],
#     # "AEO": [2023],
#     "NoNewGasYear": [2050],
#     "MaxWind": [1000],
#     # "FlexReserveReq": [0.06],
# }
# combinations += [dict(zip(addl_params.keys(), v)) for v in itertools.product(*addl_params.values())]
combinations

In [ ]:
from joblib import Parallel, delayed
import subprocess

folders = []
for i, combo in enumerate(combinations): 
    logger.info(f"Saving case: {i+1} of {len(combinations)}")
    folders.append(update_params_and_save_case(params=combo, wb=wb))

logger.info("Running GenX cases in parallel...")
Parallel(n_jobs=6, backend="multiprocessing")(delayed(subprocess.run)(["julia", "--project=.", "Run.jl", base_folder]) for base_folder in folders)

for folder in folders:
    !chmod -R 777 ./
    load_case_results(wb=wb, base_folder=UPath(folder))

In [ ]:
folders

In [ ]:
for folder in [UPath("/Users/roderick/PycharmProjects/resilient-transition/GenX.jl/cases/2025-06-05/KRC-6Years-24Days-NoPlan-Retire-BBB-NoGasAfter2040-Wind1GW")]:
    # Load portfolio into spreadsheet
    load_case_results(wb=wb, base_folder=folder)
    
    # Save PCM cases
    wb.sheets["GenX Settings"].range("EnablePCM").value = True
    wb.sheets["GenX Settings"].range("PCMPortfolio").value = str(folder)

    pcm_folder = save_multistage_case(wb=wb)
    subprocess.run(["julia", "--project=.", "Run.jl", pcm_folder])
    wb.sheets["GenX Settings"].range("EnablePCM").value = False

In [ ]:
for folder in folders:
    df = pd.read_csv(folder / "results" / "capacities_multi_stage.csv")
    df = df.loc[df["Resource"].str.contains("Battery"), ["Resource", "EndCap_p6"]]
    print(folder.stem, df)

# PCM

In [ ]:
portfolio_to_load = UPath("/Users/roderick/PycharmProjects/resilient-transition/GenX.jl/cases/2025-05-27/KRC-26Years-24Days-NoPlan-Retire-Build-Fuels-BBB-PRM-MinCF-0-1GWRate")

In [ ]:
# Load portfolio into spreadsheet
load_case_results(wb=wb,base_folder=portfolio)
df = pd.read_csv(portfolio / "planning_periods.csv").iloc[:, 1].squeeze()
# wb.sheets["GenX Settings"].range("ModeledYears[Modeled]").options(transpose=True).value = df.values

In [ ]:
# Save PCM cases
wb.sheets["GenX Settings"].range("EnablePCM").value = True
wb.sheets["GenX Settings"].activate()
run_and_load_case(wb=wb)
wb.sheets["GenX Settings"].range("EnablePCM").value = False

In [ ]:
wb = xw.Book(
    "/Users/roderick/Library/CloudStorage/OneDrive-SharedLibraries-ResilientTransition/5.001 Kentucky Resource Council - Documents/Data/Kentucky Load Resource Model.xlsb"
)
type_map = wb.sheets["GenX Resources"].range("TypeMap").options(pd.DataFrame, index=0).value.dropna().set_index("resource").squeeze(axis=1).to_dict()

# TDR Clustering Visualization

- [ ] Plots
- [ ] Are TDRs the same for every planning period? If not, how much are they changing?

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

axes = dict(
    showgrid=False,
    linecolor="rgb(120, 120, 120)",
    linewidth=1,
    showline=True,
    ticks="outside",
    tickcolor="rgb(120, 120, 120)",
    mirror=True,
)

pio.templates["e3"] = go.layout.Template(
    layout=go.Layout(
        font=dict(family="CommitMono", size=11, color="rgb(120, 120, 120)"),
        title=dict(
            font=dict(
                # size=32,
                color="rgb(3, 78, 110)",
            ),
            x=0.05,
            y=0.95,
            xanchor="left",
            yanchor="bottom",
        ),
        xaxis=axes,
        yaxis=axes,
        margin=dict(t=60, b=100, r=60, l=60),
    )
)

pio.templates["5.4x12.32"] = go.layout.Template(
    layout=go.Layout(
        height=5.4 * 144,
        width=12.32 * 144,
    )
)

pio.templates.default = "e3"

In [ ]:
import yaml
import pandas as pd
from upath import UPath
from plotly.subplots import make_subplots
import plotly.graph_objects as go

base_path = base_folder / "inputs" / "inputs_p1"
timeseries_to_compare = {
    "Demand_data.csv": [("Demand_MW_z1", "rgba(54, 176, 72, 0.5)")],
    "Generators_variability.csv": [
        ("Solar:0", "rgba(255, 192, 0, 0.5)"),
        ("Wind:0", "rgba(49, 235, 255, 0.5)"),
        ("Wind - New Generic:0", "rgba(49, 235, 255, 0.5)"),
        ("Wind - New Generic:1", "rgba(49, 235, 255, 0.5)"),
        ("Wind - New Generic:2", "rgba(49, 235, 255, 0.5)"),
        ("Wind - New Generic:3", "rgba(49, 235, 255, 0.5)"),
        ("Solar - New Generic:0", "rgba(255, 192, 0, 0.5)"),
        ("Solar - New Generic:1", "rgba(255, 192, 0, 0.5)"),
        ("Solar - New Generic:2", "rgba(255, 192, 0, 0.5)"),
        ("Solar - New Generic:3", "rgba(255, 192, 0, 0.5)"),
    ],
}


def tdr_plots(base_path: UPath, timeseries_to_compare: dict[str, list[str]]):
    # Get TDR settings & period mapping
    with open(base_path / "TDR_results" / "time_domain_reduction_settings.yml", "r") as f:
        tdr_settings = yaml.load(f, Loader=yaml.SafeLoader)

    period_map = pd.read_csv(base_path / "TDR_results" / "Period_map.csv", dtype=int)

    with open(base_path / "tdr_plots.html", "w") as tdr_plots_file:
        # Get timeseries to plot
        for i, (csv_file, columns) in enumerate(timeseries_to_compare.items()):
            for column, color_str in columns:
                logger.info(f"Plotting {csv_file}: {column}")
                df = pd.read_csv(base_path / "system" / csv_file)[["Time_Index", column]]

                df["Period_Index"] = ((df["Time_Index"] - 1) // tdr_settings["TimestepsPerRepPeriod"]) + 1
                df["Hour"] = (df["Time_Index"]) - ((df["Period_Index"] - 1) * tdr_settings["TimestepsPerRepPeriod"])
                df["Rep_Period"] = df.merge(period_map, on="Period_Index")["Rep_Period"]

                df = df.merge(
                    df[["Period_Index", "Hour", column]],
                    left_on=["Rep_Period", "Hour"],
                    right_on=["Period_Index", "Hour"],
                    suffixes=["_original", "_sampled"],
                )[[f"{column}_original", f"{column}_sampled"]]
                df.index = pd.Timestamp("1/1/2007") + pd.to_timedelta(df.index, unit="h")

                # Plot
                fig = make_subplots(
                    rows=2,
                    cols=1,
                    subplot_titles=["Chronological", "Duration Curve"],
                    vertical_spacing=0.15,
                )
                fig.update_layout(
                    title_text=f"Time Domain Reduction Comparison:<br><b>{column}",
                    legend_tracegroupgap=180,
                    width=1000,
                    height=500,
                )

                # Chronological
                fig.add_trace(
                    go.Scatter(
                        x=df.index,
                        y=df[f"{column}_original"],
                        name="Original",
                        legendgroup=1,
                        line=dict(
                            color="rgba(20, 20, 20, 0.5)",
                            width=1,
                        ),
                    ),
                    row=1,
                    col=1,
                )
                fig.add_trace(
                    go.Scatter(
                        x=df.index,
                        y=df[f"{column}_sampled"],
                        name="Sampled",
                        legendgroup=1,
                        line=dict(
                            color=color_str,
                            width=1,
                        ),
                    ),
                    row=1,
                    col=1,
                )

                # Duration curve
                fig.add_trace(
                    go.Scatter(
                        y=df[f"{column}_original"].sort_values(ascending=False, ignore_index=True),
                        name="Original",
                        legendgroup=2,
                        line=dict(
                            color="rgba(20, 20, 20, 0.5)",
                            width=1,
                        ),
                    ),
                    row=2,
                    col=1,
                )
                fig.add_trace(
                    go.Scatter(
                        y=df[f"{column}_sampled"].sort_values(ascending=False, ignore_index=True),
                        name="Sampled",
                        legendgroup=2,
                        line=dict(
                            color=color_str,
                            width=1,
                        ),
                    ),
                    row=2,
                    col=1,
                )
                tdr_plots_file.write(fig.to_html(full_html=False, include_plotlyjs="cdn" if i == 0 else None))
        logger.info(f"Interactive plots saved: {base_path / 'tdr_plots.html'}")


tdr_plots(base_path, timeseries_to_compare)

# Temperature Investigation

Seeing how ERA5 2m dewpoint temperature for Lexington (lat 38, lon -84.5) varies over years

In [ ]:
df = pd.read_csv(UPath("/Users/roderick/Downloads/reanalysis-era5-single-levels-timeseries-sfcd_aowpko.csv"), index_col=0, parse_dates=True)[["d2m"]]
df = ((df - 273.15) * 1.8) + 32

In [ ]:
full = df.loc[(df.index.year>=1960) & (df.index.year<=2020)]
full = full.squeeze().copy(deep=True).sort_values(ascending=False, ignore_index=True)
full.index = full.index / len(full)
full

In [ ]:
shorter_range = (2007, 2010)
shorter = df.loc[(df.index.year>=shorter_range[0]) & (df.index.year <= shorter_range[1])]
shorter = shorter.squeeze().copy(deep=True).sort_values(ascending=False, ignore_index=True)
shorter.index = shorter.index / len(shorter)
shorter

In [ ]:
import plotly.graph_objects as go

go.Figure(
    data=[
        go.Scatter(x=full.index, y=full, name="1960-2020"),
        go.Scatter(x=shorter.index, y=shorter, name="-".join(map(str, shorter_range))),
    ]
)

In [ ]:
df[["year", "month", "day", "hour"]] = list(zip(df.index.year, df.index.month, df.index.day, df.index.hour))
df = pd.pivot_table(df, values="d2m", columns=["year"], index=["month", "day", "hour"])
df.max(axis=0).plot()

In [ ]:
import plotly.express as px
px.line(df.reset_index(drop=True))

In [ ]:
df.max(axis=0)